In [1]:
import pandas as pd
import time
import datetime
import redis
from flask import current_app
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
#!/usr/bin/python
import MySQLdb

db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     #passwd="xpx",  # your password
                     db="xudatabase")        # name of the data base

cur = db.cursor()

In [3]:
# import article data from mysql db 
def import_data_articles(table_name):
    #ds = pd.read_csv(data_source)
    ds = cur.execute('select * from ' +table_name)
    r = cur.fetchall()
    return pd.DataFrame(list(r),columns=['PMID', 'abstract'])
    

In [4]:
userId =1
userpool = pd.read_csv("./data/testdata.csv")
cur.execute('select pmid, abstract from user_articles where user_id=%d'% (userId))
r = cur.fetchall()
userpool= pd.DataFrame(list(r),columns=['PMID', 'abstract'])
print userpool
print userpool.shape



   PMID                                 abstract
0    -1  blood cancer cell experiment yeast test
(1, 2)


In [5]:
dailypool = import_data_articles('articles')
print dailypool.head()

       PMID                                           abstract
0  28288257  Importance: X-linked adrenoleukodystrophy (ALD...
1  28288262  Importance: The heterogeneity of rate of clini...
2  28288264  Purpose: To evaluate morselized amniotic membr...
3  28288272  BACKGROUND: Asthma is a common chronic disease...
4  28288273  Two-dimensional materials offer a remarkably r...


In [15]:
# refactor the training logic into this function
def train(userpool, dailypool):
    ds = pd.concat([userpool, dailypool])
    ds.reset_index(drop=True, inplace=True)
    
    #clean up not ASCII chars
    ds['abstract'] = ds["abstract"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
    
    tf = TfidfVectorizer(analyzer='word',
                             ngram_range=(1, 3),
                             min_df=0,
                             stop_words='english')
    tfidf_matrix = tf.fit_transform(ds['abstract'])
    print tfidf_matrix.shape
    cosine_similarities = linear_kernel(tfidf_matrix[:userpool.shape[0]], tfidf_matrix[userpool.shape[0]:])
    similarities = pd.DataFrame(cosine_similarities)
    
    prediction_matrix = pd.DataFrame(data=userpool.PMID, columns=['PMID'], index=userpool.index)
    similarity_scores = prediction_matrix.copy()
    
    top_n = 12
    if (ds.shape[0] < top_n):
        top_n = ds.shape[0] + 1

    for i in range(1,top_n-1):
        col_name = i
        prediction_matrix[col_name] = None
        similarity_scores[col_name] = -1

    now = datetime.datetime.now()
    
    for idx in range(userpool.shape[0]):
        print "idx: %s" % idx
        similar_indices = cosine_similarities[idx].argsort()[:-top_n:-1]
        print "similar indices: %s" % similar_indices
        similar_items = [(cosine_similarities[idx][i], ds['PMID'][i]) for i in similar_indices]
        print "similar items: \n %s" % similar_items
        for i in range(1,top_n-1): 

            prediction_matrix.ix[idx, i] = similar_items[i][1] # insert into db directly 
            similarity_scores.ix[idx, i] = similar_items[i][0] # insert into a db table with date

        query = 'insert into recommend_articles values ("%s", %d, %d,\
            %d, %d, %d, %d, %d,\
            %d, %d, %d, %d, %d,\
            %.6f,%.6f,%.6f,%.6f,%.6f,\
            %.6f,%.6f,%.6f,%.6f,%.6f)'%(now, userId,userpool.ix[idx,'PMID'],\
                                                         similar_items[0][1],similar_items[1][1],similar_items[2][1],\
                                                         similar_items[3][1],similar_items[4][1],similar_items[5][1],\
                                                         similar_items[6][1],similar_items[7][1],similar_items[8][1],\
                                                         similar_items[9][1],similar_items[0][0],similar_items[1][0],\
                                                         similar_items[2][0],\
                                                         similar_items[3][0],similar_items[4][0],similar_items[5][0],\
                                                         similar_items[6][0],similar_items[7][0],similar_items[8][0],\
                                                         similar_items[9][0])

        cur.execute(query)
        db.commit()
    return 1;

In [17]:
start = time.time()
train(userpool, dailypool)
print("Engine trained in %s seconds." % (time.time() - start))

(214, 3457)
idx: 0
similar indices: [ 13  83 113 129 154 106 179 187 209 134  97]
similar items: 
 [(0.064928788941372884, 28288281), (0.046140608908230005, 28288362), (0.045955454114953026, 28288393), (0.042212439846261419, 28288410), (0.041913581605979654, 28288435), (0.040073906900026601, 28288386), (0.040023343172268751, 28288460), (0.037010885075714595, 28288470), (0.036333615501071378, 28288496), (0.033475954211570472, 28288415), (0.0, 28288377)]
Engine trained in 0.0718228816986 seconds.


In [ ]:
db.close()